In [ ]:
pip install spotipy
pip install matplotlib
pi install pandas
pip install scikit-learn
pip install pydotplus
##installing required packages

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util


cid ='4a8f691d58c34c059a235249fa877f6e' # Client ID; copy this from your app created on beta.developer.spotify.com
secret = '9b9a8e97d24c4688abb5f188a2d80de3' # Client Secret; copy this from your app
username = '31obxjrwpdtksnjtof7oopbelnwq' # 
#for avaliable scopes see https://developer.spotify.com/web-api/using-scopes/
scope = 'user-library-read playlist-modify-public playlist-read-private'

redirect_uri='http://localhost:3000/callback'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

token = util.prompt_for_user_token(username, scope, cid, secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [ ]:
import pandas as pd
playlistCreatorUserName='UserName of user who created the playlist'
sourcePlaylistID = 'PlaylistId'
sourcePlaylist = sp.user_playlist(playlistCreatorUserName, sourcePlaylistID);
tracks = sourcePlaylist["tracks"];
songs = tracks["items"];

track_ids = []
track_names = []

for i in range(0, len(songs)):
    if songs[i]['track']['id'] != None: 
        track_ids.append(songs[i]['track']['id'])
        track_names.append(songs[i]['track']['name'])

features = []
for i in range(0,len(track_ids)):
    audio_features = sp.audio_features(track_ids[i])
    for track in audio_features:
        features.append(track)
        
playlist_df = pd.DataFrame(features, index = track_names)

In [ ]:
playlist_df.head() ##ipo ee data frame il playlist full kitum namak

In [ ]:

playlist_df=playlist_df[["id", "acousticness", "danceability", "duration_ms", 
                         "energy", "instrumentalness",  "key", "liveness",
                         "loudness", "mode", "speechiness", "tempo", "valence"]]
playlist_df.head() ##enit avishyam ullla colums mattram edukum 

In [ ]:
import numpy as np
N = 30
K = 15 # K zeros, N-K ones
arr = np.array([1] * K + [0] * (N-K))


In [ ]:
playlist_df['isLike']=arr ##ennit data frame il new colum ayit athina add akunnu array na

In [ ]:
playlist_df.head() ##ipo data frame il last column is islike vannu kanum

In [ ]:
playlist_df=playlist_df.sort_values(by='id', ascending=False)
playlist_df.head(10) ## id vechit sort chyunnu 

In [ ]:
playlist_df['isLike'][0:23].values.tolist()

In [ ]:
from sklearn.model_selection import train_test_split  
## ithann machine learning module import chyunnath

In [ ]:
X= playlist_df.drop(['id', 'isLike'], axis=1) 
Y=playlist_df['isLike']

In [ ]:
x_train, x_test, y_train, y_test= train_test_split(X, Y, test_size= 0.25, random_state=0)  


In [ ]:
from sklearn.preprocessing import StandardScaler    
st_x= StandardScaler()    
x_train= st_x.fit_transform(x_train)    
x_test= st_x.transform(x_test)  

In [ ]:
from sklearn.neighbors import KNeighborsClassifier  
classifier= KNeighborsClassifier(n_neighbors=4, metric='minkowski', p=2 )  
classifier.fit(x_train, y_train) 

In [ ]:
y_pred= classifier.predict(x_test)  

In [ ]:
from sklearn.metrics import confusion_matrix  
cm= confusion_matrix(y_test, y_pred) 

In [ ]:
cm

In [ ]:
from sklearn import metrics

print(metrics.accuracy_score(y_test, y_pred))

In [ ]:
rec_tracks = []
for i in playlist_df[playlist_df['isLike']==1]['id'].values.tolist():
    rec_tracks += sp.recommendations(seed_tracks=[i], limit=5, max_instrumentalness=max(playlist_df['instrumentalness']), min_instrumentalness=min(playlist_df['instrumentalness']),max_tempo=max(playlist_df['tempo']),min_tempo=min(playlist_df['tempo']))['tracks'];

rec_track_ids = []
rec_track_names = []
for i in rec_tracks:
    rec_track_ids.append(i['id'])
    rec_track_names.append(i['name'])

rec_features = []
for i in range(0,len(rec_track_ids)):
    rec_audio_features = sp.audio_features(rec_track_ids[i])
    for track in rec_audio_features:
        rec_features.append(track)
        
rec_playlist_df = pd.DataFrame(rec_features, index = rec_track_ids)
rec_playlist_df.head() 

In [ ]:
rec_playlist_df=rec_playlist_df[["id","acousticness", "danceability", "duration_ms", 
                         "energy", "instrumentalness",  "key", "liveness",
                         "loudness", "mode", "speechiness", "tempo", "valence"]]

In [ ]:
x_final= rec_playlist_df[["acousticness", "danceability", "duration_ms", 
                         "energy", "instrumentalness",  "key", "liveness",
                         "loudness", "mode", "speechiness", "tempo", "valence"]]

In [ ]:
rec_playlist_df_scaled = StandardScaler().fit_transform(x_final)

In [ ]:
rec_playlist_df_scaled

In [ ]:
y_pred=classifier.predict(rec_playlist_df_scaled)

In [ ]:
rec_playlist_df['isLike']=np.array(y_pred)

In [ ]:
recs_to_add =rec_playlist_df[rec_playlist_df['isLike']==1]['id'].values.tolist()

In [ ]:
playlist_recs = sp.user_playlist_create(playlistCreatorUserName, 
                                        name='KNN Playlist')

In [ ]:
len(recs_to_add)

In [ ]:
sp.user_playlist_add_tracks(playlistCreatorUserName, playlist_recs['id'], recs_to_add);

In [ ]:
##Descicion Tree

In [ ]:
from sklearn import tree
import pydotplus
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt
import matplotlib.image as pltimg

In [ ]:
features=["acousticness", "danceability", "duration_ms", 
                         "energy", "instrumentalness",  "key", "liveness",
                         "loudness", "mode", "speechiness", "tempo", "valence"]

In [ ]:
dtree = DecisionTreeClassifier()
dtree = dtree.fit(X, Y)
data = tree.export_graphviz(dtree, out_file=None, feature_names=features)
graph = pydotplus.graph_from_dot_data(data)
graph.write_png('mydecisiontree.png')

img=pltimg.imread('mydecisiontree.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
from sklearn import neighbors 
from mlxtend.plotting import plot_decision_regions
import matplotlib.pyplot as plt


In [ ]:
x_plot = playlist_df[['acousticness','danceability']][0:22].values
plot = np.array([0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1])
k=4
clf = neighbors.KNeighborsClassifier(n_neighbors=k)
clf.fit(x_plot, y_plot)
# Plotting decision region
plot_decision_regions(x_plot, y_plot, clf=clf, legend=2)
# Adding axes annotations
plt.xlabel("acousticness")
plt.ylabel("danceability")
plt.title("Knn with K="+ str(k))
plt.show()
